<a href="https://colab.research.google.com/github/CE334/CE334_200633_Nikhil_bhardwaj/blob/main/Ce334_Nikhil_Bhardwaj__200633_lab3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Reading the data

In [3]:


import numpy as np
import pandas as pd

# Step 1: Read data


file_path = "/content/drive/MyDrive/Colab Notebooks/datag1.csv"
df = pd.read_csv(file_path)

In [10]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datag1.csv", skiprows=20)

# Remove "/" from column names
df.columns = df.columns.str.replace("/", "")

print(df.columns)


Index(['Station', 'Date', 'Time', 'CorrGrav', 'Line', 'StdDev', 'StdErr',
       'RawGrav', 'X', 'Y', 'SensorTemp', 'TideCorr', 'TiltCorr', 'TempCorr',
       'DriftCorr', 'MeasurDur', 'InstrHeight', 'LatUser', 'LonUser',
       'ElevUser', 'LatGPS', 'LonGPS', 'ElevGPS',
       'Corrections[drift-temp-na-tide-tilt]'],
      dtype='object')


STEP 2: Converting Time to Decimal Hours

In [15]:

# STEP 2: CONVERT TIME TO DECIMAL HOURS
def time_to_decimal(t):
    h, m, s = map(float, t.split(":"))
    return h + m/60 + s/3600

df["Time_decimal"] = df["Time"].apply(time_to_decimal)


STEP 3 and 4: Form Difference Observations

In [62]:

# STEP 3: USING ALL OBSERVATIONS

# Reference observation = first reading
ref_grav = df.loc[0, "CorrGrav"]
ref_time = df.loc[0, "Time_decimal"]

df["Δy"] = df["CorrGrav"] - ref_grav
df["Δt"] = df["Time_decimal"] - ref_time

# Removing first observation (datum)
obs = df.iloc[1:].copy()




In [63]:
# STEP 4: FORM DIFFERENCE OBSERVATIONS (Eq. 2)
# Reference = First Station
ref_grav = station_avg.loc[0, "CorrGrav"]
ref_time = station_avg.loc[0, "Time_decimal"]

station_avg["Δy"] = station_avg["CorrGrav"] - ref_grav
station_avg["Δt"] = station_avg["Time_decimal"] - ref_time

# Remove reference row
obs = station_avg.iloc[1:].copy()

STEP 5: Constructing the Design matrix (A)

In [65]:

# STEP 5: DESIGN MATRIX (A)

stations = df["Station"].unique()
fixed_station = stations[0]

# creating station index excluding reference station
station_dict = {s:i for i,s in enumerate(stations) if s != fixed_station}

n_obs = len(obs)
n_params = len(stations)

A = np.zeros((n_obs, n_params))

for i in range(n_obs):
    station = obs.iloc[i]["Station"]
    t = obs.iloc[i]["Δt"]

    if station != fixed_station:
        col = station_dict[station]
        A[i, col] = 1


    A[i, -1] = t

print("\nDesign Matrix A:")
print(A)
print("Rank of A:", np.linalg.matrix_rank(A))



Design Matrix A:
[[ 0.          0.          0.          0.          0.          0.
   1.          0.          0.          0.17277778]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.36138889]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.         -0.44361111]
 [ 0.          0.          1.          0.          0.          0.
   0.          0.          0.         -0.19722222]
 [ 0.          0.          0.          0.          0.          0.
   0.          1.          0.          0.25777778]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.45      ]
 [ 0.          1.          0.          0.          0.          0.
   0.          0.          0.         -0.29916667]
 [ 0.          0.          0.          1.          0.          0.
   0.          0.          0.         -0.095     ]
 [ 0.          0.          0.          0.     

Step 6: Weight matrix

In [66]:
# STEP 6: WEIGHT MATRIX (P)
# Using StdDev from CG-6 file
sigma = obs["StdDev"].values
P = np.diag(1 / sigma**2)

print("\nWeight Matrix P:")
print(P)



Weight Matrix P:
[[5459.97233614    0.            0.            0.            0.
     0.            0.            0.            0.        ]
 [   0.         7022.25273868    0.            0.            0.
     0.            0.            0.            0.        ]
 [   0.            0.         7831.46683374    0.            0.
     0.            0.            0.            0.        ]
 [   0.            0.            0.         4800.28161652    0.
     0.            0.            0.            0.        ]
 [   0.            0.            0.            0.         4959.33346558
     0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
  4309.33353763    0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.         6009.25425155    0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.         4504.30160804    0.

STEP 7: OBSERVATION VECTOR L

In [67]:
# STEP 7: OBSERVATION VECTOR (L)
L = obs["Δy"].values.reshape(-1,1)

STEP 8: NORMAL EQUATIONS

In [57]:
# STEP 8: NORMAL EQUATIONS
N = A.T @ P @ A
U = A.T @ P @ L

print("\nNormal Matrix N:")
print(N)

print("\nVector U:")
print(U)



Normal Matrix N:
[[    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.          6009.25425155     0.             0.
      0.             0.             0.             0.
      0.         -1797.76856359]
 [    0.             0.          4800.28161652     0.
      0.             0.             0.             0.
      0.          -946.7222077 ]
 [    0.             0.             0.          4504.30160804
      0.             0.             0.             0.
      0.          -427.90865276]
 [    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.             0.
      0.          5541.56481476     0.             0.
      0.           472.57233281]
 [    0.             0.             0.             0.
      0.             0.          5459.97233614     0.
      0.           9

STEP 9: Solving least squares

In [68]:
# STEP 9: SOLVE LEAST SQUARES
X = np.linalg.pinv(N) @ U

print("\nEstimated Parameters:")

for i in range(n_params - 1):
    print(f"Δg_station_{i+1} = {X[i,0]:.6f} mGal")

print(f"Drift d = {X[-1,0]:.6f} mGal/hour")




Estimated Parameters:
Δg_station_1 = 0.000000 mGal
Δg_station_2 = -0.165758 mGal
Δg_station_3 = 2.701911 mGal
Δg_station_4 = 1.443702 mGal
Δg_station_5 = 0.000000 mGal
Δg_station_6 = 2.865044 mGal
Δg_station_7 = 1.362327 mGal
Δg_station_8 = -0.139050 mGal
Δg_station_9 = 2.846981 mGal
Drift d = -7.877520 mGal/hour


STEP 10: Residuals

In [69]:
# STEP 10: Residuals
V = A @ X - L

print("\nResiduals:")
print(V)


Residuals:
[[-3.77475828e-15]
 [-3.10862447e-15]
 [-7.68944751e-01]
 [ 1.77635684e-15]
 [-2.22044605e-15]
 [-1.37758385e+00]
 [ 3.55271368e-15]
 [-8.88178420e-16]
 [-3.55271368e-15]]


Step 11 : Orthogonality check

In [70]:
# STEP 11: ORTHOGONALITY CHECK (Aᵀ P V ≈ 0)
print("\nOrthogonality Check (Aᵀ P V):")
print(A.T @ P @ V)


Orthogonality Check (Aᵀ P V):
[[ 0.00000000e+00]
 [ 2.13491598e-11]
 [ 8.52701308e-12]
 [-4.00062348e-12]
 [ 0.00000000e+00]
 [-1.96875931e-11]
 [-2.06100758e-11]
 [-1.10119324e-11]
 [-2.18295467e-11]
 [-1.89593135e-11]]


Step 12 : Getting Redundancy and no of parameters

In [71]:

# STEP 12: A-POSTERIORI REFERENCE VARIANCE
# Residuals
V = A @ X - L

# Redundancy
r = n_obs - n_params

print("\nNumber of observations (n):", n_obs)
print("Number of parameters (u):", n_params)
print("Redundancy (r):", r)

# A-posteriori variance factor
sigma0_post = (V.T @ P @ V) / r

print("\nA-posteriori reference variance:")
print(sigma0_post[0,0])



Number of observations (n): 9
Number of parameters (u): 10
Redundancy (r): -1

A-posteriori reference variance:
-12808.541417960336


STEP 13: VARIANCE-COVARIANCE MATRIX

In [76]:
# STEP 13: VARIANCE-COVARIANCE MATRIX
Cx = sigma0_post[0,0] * np.linalg.pinv(N)

print("\nVariance-Covariance Matrix of Estimated Parameters:")
print(Cx)



Variance-Covariance Matrix of Estimated Parameters:
[[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -2.60639396e+00 -3.13088632e-01 -1.50811707e-01
   4.85684023e-17  1.35377761e-01  2.74283281e-01  4.09220072e-01
   5.73701846e-01 -1.58749166e+00]
 [-0.00000000e+00 -3.13088632e-01 -2.87468970e+00 -9.94209028e-02
  -4.42782349e-17  8.92462490e-02  1.80818133e-01  2.69773678e-01
   3.78206417e-01 -1.04653582e+00]
 [-0.00000000e+00 -1.50811707e-01 -9.94209028e-02 -2.89151435e+00
  -9.64853878e-17  4.29890382e-02  8.70983120e-02  1.29947321e-01
   1.82178302e-01 -5.04105986e-01]
 [-0.00000000e+00  8.44150072e-16  6.83564371e-16  1.37466558e-16
  -4.83782877e-33 -5.45164457e-18 -8.55367633e-16 -5.44541865e-16
  -9.73855378e-16  2.06174077e-15]
 [-0.00000000e+00  1.35377761e-01  8.92462490e-02  4.29890382e-02
  -2.63763381e-16 -2.34994780e+00 -7.81847421e

Step 14 : Adjusted gravity values

In [75]:

# STEP 14: FINAL ADJUSTED GRAVITY VALUES
# Extract estimated parameters
drift = X[-1, 0]  # last parameter = drift

# Create dictionary of estimated gravity corrections
estimated_grav = {}

for station, idx in station_dict.items():
    estimated_grav[station] = X[idx, 0]

# Reference station gravity (fixed)
ref_station = fixed_station
ref_gravity = ref_grav

# Compute adjusted gravity for each station
adjusted_results = []

for station in df["Station"].unique():

    if station == ref_station:
        adjusted_value = ref_gravity
    else:
        delta_g = estimated_grav[station]
        adjusted_value = ref_gravity + delta_g

    adjusted_results.append([station, adjusted_value])

adjusted_df = pd.DataFrame(adjusted_results, columns=["Station", "AdjustedGravity"])

print("\nFinal Adjusted Gravity Values (Station-wise):")
print(adjusted_df)



Final Adjusted Gravity Values (Station-wise):
  Station  AdjustedGravity
0     FF1      2907.452100
1     TF1      2907.286342
2     FF2      2910.154011
3     TF2      2908.895802
4     5F1      2907.452100
5     TF3      2910.317144
6     5F2      2908.814427
7     SF1      2907.313050
8     5F3      2910.299081
9     SF2      2899.574580
